In [1]:
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import *

In [6]:
key = os.environ.get('AZURE_SEARCH_KEY', '596c8282d719425786ef371493c5e6b8')
bear_types = ['grizzly','black','teddy']
path = Path('bears')

In [7]:
results = search_images_bing(key, 'grizzly bears')
ims = results.attrgot('contentUrl')
len(ims)

150

In [9]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [10]:
fns = get_image_files(path)
fns

(#413) [Path('bears/teddy/899585d4-7da9-4a41-8534-d8da69b477c1.jpeg'),Path('bears/teddy/a739010f-516e-4c1f-80f5-654376c5fb76.jpeg'),Path('bears/teddy/e7db18eb-c66d-4c17-a0a8-31475ca1c9b6.jpg'),Path('bears/teddy/b22560a8-1cb1-4df2-8df7-71b99ef0e9c7.jpeg'),Path('bears/teddy/c801187c-bab9-4fce-993c-ffc7192586af.jpg'),Path('bears/teddy/fe1f66fc-3f1d-48c2-8c10-10d6420a148e.jpg'),Path('bears/teddy/2ac7dcac-2d13-4479-910f-7d0d75c07bb2.png'),Path('bears/teddy/bc56003e-aa70-4cd9-a1cc-58e53970d3cc.jpeg'),Path('bears/teddy/8128988f-2ca9-4dcf-b99d-e8477c15dc79.jpg'),Path('bears/teddy/c0642fd5-020c-4641-a9b6-3a4ae12d319c.jpg')...]

In [11]:
failed = verify_images(fns)
failed.map(Path.unlink);
bears = DataBlock(blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))
bears = bears.new(item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

epoch,train_loss,valid_loss,error_rate,time
0,1.484172,0.065365,0.012195,00:17


epoch,train_loss,valid_loss,error_rate,time
0,0.174430,0.071224,0.012195,00:18
1,0.133014,0.112425,0.012195,00:16
2,0.110064,0.120381,0.012195,00:18
3,0.097502,0.116101,0.012195,00:17


In [12]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner(path/'export.pkl')

In [16]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [17]:
img = PILImage.create(btn_upload.data[-1])

In [18]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

Output()

In [19]:
pred,pred_idx,probs = learn_inf.predict(img)

In [22]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: grizzly; Probability: 1.0000')

In [23]:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [24]:
btn_upload = widgets.FileUpload()

In [25]:
btn_run = widgets.Button(description='Classify')
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [26]:
VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])

In [27]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 33.6 MB/s eta 0:00:0000:0100:01
Config option `kernel_spec_manager_class` not recognized by `EnableServerExtensionApp`.
Enabling: voila
- Writing config: /opt/conda/etc/jupyter
    - Validating...
      voila 0.5.5 OK


NameError: name 'voila' is not defined